# Jupyter Notebook for Training a Language Model with Unsloth

This Jupyter Notebook demonstrates the process of setting up and training a language model using the Unsloth library. The notebook covers the following steps:

1. **Environment Setup**:
    - Create a conda environment using the `environment.yml` file.

2. **Model Initialization**:
    - Import necessary libraries and set up configurations for the language model.
    - Load the pre-trained model and tokenizer from Unsloth.

3. **Model Configuration**:
    - Apply Parameter-Efficient Fine-Tuning (PEFT) to the model using LoRA (Low-Rank Adaptation).

4. **Chat Template Application**:
    - Use a chat template to format the input data for the model.
    - Load and preprocess the dataset using the `datasets` library.

5. **Training Setup**:
    - Initialize the `SFTTrainer` with the model, tokenizer, and training dataset.
    - Define training arguments such as batch size, learning rate, and optimization settings.

6. **Model Training**:
    - Train the model using the `SFTTrainer` and capture training statistics.

7. **Model Saving**:
    - Save the trained model locally for future use.

This notebook provides a comprehensive guide to fine-tuning a language model with Unsloth, making it suitable for various natural language processing tasks.


In [ ]:
# setting up the environment
!conda env create -f environment.yml

In [ ]:
#  model setup
from unsloth import FastLanguageModel
import torch
import os

#  fetching the dataset
dataset = os.getenv("DATASET", "yahma/alpaca-cleaned")
max_seq_length = 2048 # Choose any! We auto support RoPE Scaling internally!
dtype = None # None for auto detection. Float16 for Tesla T4, V100, Bfloat16 for Ampere+
load_in_4bit = True # Use 4bit quantization to reduce memory usage. Can be False.

# setting the template for the model
unsloth_template = \
    "{{ bos_token }}"\
    "{{ 'You are a helpful assistant to the user\n' }}"\
    "{% for message in messages %}"\
        "{% if message['role'] == 'user' %}"\
            "{{ '>>> User: ' + message['content'] + '\n' }}"\
        "{% elif message['role'] == 'assistant' %}"\
            "{{ '>>> Assistant: ' + message['content'] + eos_token + '\n' }}"\
        "{% endif %}"\
    "{% endfor %}"\
    "{% if add_generation_prompt %}"\
        "{{ '>>> Assistant: ' }}"\
    "{% endif %}"

In [ ]:
# creating the base model
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/llama-3-8b-Instruct-bnb-4bit", # Choose ANY! eg teknium/OpenHermes-2.5-Mistral-7B
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
    # token = "hf_...", # use one if using gated models like meta-llama/Llama-2-7b-hf
)

==((====))==  Unsloth 2024.8: Fast Llama patching. Transformers = 4.44.0.
   \\   /|    GPU: Tesla T4. Max memory: 14.748 GB. Platform = Linux.
O^O/ \_/ \    Pytorch: 2.3.1+cu121. CUDA = 7.5. CUDA Toolkit = 12.1.
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.26.post1. FA2 = False]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors:   0%|          | 0.00/5.70G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/220 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/51.1k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/345 [00:00<?, ?B/s]

In [ ]:
# preaching the weights
model = FastLanguageModel.get_peft_model(
    model,
    r = 16,
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],
    lora_alpha = 16,
    lora_dropout = 0, # Supports any, but = 0 is optimized
    bias = "none",    # Supports any, but = "none" is optimized
    # [NEW] "unsloth" uses 30% less VRAM, fits 2x larger batch sizes!
    use_gradient_checkpointing = "unsloth", # True or "unsloth" for very long context
    random_state = 3407,
    use_rslora = False,  # We support rank stabilized LoRA
    loftq_config = None, # And LoftQ
)

Unsloth 2024.8 patched 32 layers with 32 QKV layers, 32 O layers and 32 MLP layers.


In [ ]:
from unsloth.chat_templates import get_chat_template

# setting up the tokenizer for the model
tokenizer = get_chat_template(
    tokenizer,
    chat_template = "llama-3", # Supports zephyr, chatml, mistral, llama, alpaca, vicuna, vicuna_old, unsloth
    mapping = {"role" : "from", "content" : "value", "user" : "human", "assistant" : "gpt"}, # ShareGPT style
)
# formatting the prompts
def formatting_prompts_func(examples):
    convos = examples["conversations"]
    texts = [tokenizer.apply_chat_template(convo, tokenize = False, add_generation_prompt = False) for convo in convos]
    return { "text" : texts, }
pass

# loading the dataset
from datasets import load_dataset
dataset = load_dataset(dataset, split = "train")
dataset = dataset.map(formatting_prompts_func, batched = True,)

Generating train split:   0%|          | 0/9033 [00:00<?, ? examples/s]

Map:   0%|          | 0/9033 [00:00<?, ? examples/s]

In [ ]:
from trl import SFTTrainer
from transformers import TrainingArguments
from unsloth import is_bfloat16_supported

# setting up the trainer
trainer = SFTTrainer(
    model = model, # Can be a list of models for ensemble training
    tokenizer = tokenizer, # Can be a list of tokenizers for ensemble
    train_dataset = dataset, # Can be a list of datasets for ensemble training
    dataset_text_field = "text", # The field in the dataset that contains the text
    max_seq_length = max_seq_length, # Choose any! We auto support RoPE Scaling internally!
    dataset_num_proc = 2, # Number of processes to use for dataset preprocessing
    packing = False, # Can make training 5x faster for short sequences.
    args = TrainingArguments(
        per_device_train_batch_size = 2, #pre device batch size for training
        gradient_accumulation_steps = 4, # setting gradient accumulation steps
        warmup_steps = 5, # 5 is optimized
        max_steps = 60,
        learning_rate = 2e-4,
        fp16 = not is_bfloat16_supported(), # Use FP16 if not using BFloat16
        bf16 = is_bfloat16_supported(), # Use BFloat16 if supported
        logging_steps = 1, # Log every 1 step
        optim = "adamw_8bit", # Supports any, but = "adamw_8bit" is optimized
        weight_decay = 0.01, # 0.01 is optimized
        lr_scheduler_type = "linear", # Supports any, but = "linear" is optimized
        seed = 223864343, 
        output_dir = "outputs",
    ),
)

Map (num_proc=2):   0%|          | 0/9033 [00:00<?, ? examples/s]

max_steps is given, it will override any value given in num_train_epochs


In [ ]:
trainer_stats = trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs = 1
   \\   /|    Num examples = 9,033 | Num Epochs = 1
O^O/ \_/ \    Batch size per device = 2 | Gradient Accumulation steps = 4
\        /    Total batch size = 8 | Total steps = 60
 "-____-"     Number of trainable parameters = 41,943,040


Step,Training Loss
1,2.327300
2,1.431800
3,1.671600
4,1.828400
5,1.723400
6,1.448000
7,1.125600
8,1.661500
9,1.715100
10,1.380100


In [ ]:
model.save_pretrained("crowd_monitoring_model") # Local saving